In [ ]:
#@title **Choose song** { run: "auto" }
 
Search = "\u05D4\u05D9\u05D5\u05DD" #@param {type:"string"}
Sort_by = "Artist" #@param ['Song','Artist']
Progression_value = "4" #@param [6,5,4,3,2,1,0,-1,-2,-3,-4,-5] 
Prog_value = int(Progression_value)
 
# Load modules
from IPython.display import Javascript,clear_output,display
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))
from google.colab import output
import uuid
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual,Layout,jslink
import ipywidgets as widgets
import copy
from pprint import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import re
import IPython
from urllib.request import Request, urlopen
import pandas as pd
import requests
from io import StringIO
try: first_run # A way to check if element exist (i wanted to run the imports only on th first time)
except NameError: first_run = True
 
if first_run==True:
  !pip install pychord -q
  #from __future__ import print_function
  first_run = False
  # Constants
  text_alignment = 'right'
  # import song list
  #------------------------------------------------------------------------------------
  orig_url='https://drive.google.com/file/d/1aDrlFu5eSabjo8CVKAOG6ljfr5OihZLq/view?usp=sharing'
  file_id = orig_url.split('/')[-2]
  dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
  text = requests.get(dwn_url)
  text.encoding = 'utf-8'
  csv_raw = StringIO(text.text)
  df = pd.read_csv(csv_raw,encoding='utf-8')
  df['אמן'] = df['אמן'].apply(lambda x: x.strip())
  df['שיר'] = df['שיר'].apply(lambda x: x.strip())
 
from pychord import Chord
# Chord_Progression function
def Chord_Progression(orig_chord=str,transpose_level=0):
  prog_chord = Chord(orig_chord)
  prog_chord.transpose(transpose_level)
  return prog_chord.chord
 
# Choose song list from spesific artist to display
def artist_songs(artist_name=str):
  song_list = df.loc[df['shir_aman'].str.contains(artist_name)]['shir_aman'].sort_values()
  song_list = list(zip(song_list,song_list.index))
  return song_list
#------------------------------------------------------------------------------------
# Choose roder of view in the select table
if Sort_by == "Song":
  df['shir_aman'] = df['שיר']+" | "+df['אמן']
else:
  df['shir_aman'] = df['אמן']+" | "+df['שיר'] 
song_tab = widgets.Select(options=artist_songs(Search),layout=Layout(width='40%', height='200px',right='True'),rows=20)
display(song_tab)   #!!!
 
class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
 
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
 
    template = """<style>
    #{callback_id} {{
      background-color: #71A7FF;
      border: none;
      color: white;
      padding: 10px 30px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      margin: 10px 10px;
      cursor: pointer;
      border-radius: 16px;
      box-shadow: 0 9px #CFDFF9;
      font-size : 20px;
    }}
 
    #{callback_id}:hover {{background-color: #5797FE}}
    #{callback_id}:active {{
      background-color: #71A7FF;
      box-shadow: 0 4px #CFDFF9;
      transform: translateY(4px);
      }}
 
    </style>
    <button id="{callback_id}">{title}</button>
            <script>
              document.querySelector("#{callback_id}").onclick = (e) => {{
                google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                e.preventDefault();
              }};
            </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html
 
class InvokeButton1(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
 
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
 
    template = """<style>
    #{callback_id} {{
      background-color: #F688FF;
      border: none;
      color: white;
      padding: 13px 42px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      margin: 10px 10px;
      cursor: pointer;
      border-radius: 16px;
      box-shadow: 0 9px #F7E0F9;
      font-size : 20px;
    }}
 
    #{callback_id}:hover {{background-color: #EF38FE}}
    #{callback_id}:active {{
      background-color: #F688FF;
      box-shadow: 0 4px #F7E0F9;
      transform: translateY(4px);
      }}
 
    </style>
    <button id="{callback_id}">{title}</button>
            <script>
              document.querySelector("#{callback_id}").onclick = (e) => {{
                google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                e.preventDefault();
              }};
            </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html
 
def on_button_clicked(b):
  #clear_output()
  #with output:
  #display(IPython.display.HTML("<marquee style='width: 30%; color: purple;'><b>✌ ...טוען שיר</b></marquee>"))
  #clear_output()
  myurl = df.iloc[song_tab.value]['לינק']
  req = Request(myurl, headers={'User-Agent': 'Mozilla/5.0'})
  webpage = urlopen(req).read()
  soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
  song = soupified.find("div", {"class": "song_block"}) #Find the nevessary tag and class which it belongs to
  header = song.find(attrs={"class":"song_block_title"})
  # Clean the HTML from ads and more unnecessary stuff
  header = header.get_text().strip().replace("אקורדים לשיר ","")
  text_alignment = 'right'
  header = f"""<h1 style="text-align: {text_alignment}">"""+header+'</h1>'
  on_button_clicked.header = header
 
  progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
  header_progression = header+progression_status
 
  delete_nodes = [{"class":"song_rating"},
                  {"class":"row"},
                  {"id":"songad"},
                  {"class":"song_block_title"},
                  {"id":"songTopAd8Td"},
                  {"class":"videosTitle"},
                  {"type":"text/javascript"},
                  {"id":"taboola-below-article-thumbnails"},
                  {"id":"downInSongTable"},
                  {"class":"chord_info"}]
                  
  for node in delete_nodes:
    for div in song.find_all(attrs=node): 
        div.decompose()
 
  # Align guitar TABS to the left
  tab_list = ''
  tab_nodes = [{"class":"tabs"}]
  for node in tab_nodes:
    for div in song.find_all(attrs=node):
      div['dir'] = 'ltr'
      div['align'] = 'left'
      #div['display'] = 'inline-block'
      #div['width'] = '300px'
      div['position'] = 'absolute'
      div['right'] = '10%'
      #div['margin-left'] = '300px'
      tab_list=tab_list+'<p'+str(div)+'</p>'
  on_button_clicked.song=song
  # Merge Header & Song(words+chords)
  full_page=header_progression+str(song)
  on_button_clicked.full_page = full_page
 #display(IPython.display.HTML(full_page))
 
  return on_button_clicked.song,on_button_clicked.header,on_button_clicked.full_page
 
def modulate_full_page(Progression_song,header):
  for elm in Progression_song.find_all('span'):
    try:
      elm.string = elm.text.replace(elm.text, Chord_Progression(orig_chord=elm.text,transpose_level=Prog_value))
    except:
      dummy = 0
  if Prog_value > 0:
    progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
    header_progression = header+progression_status
  elif Prog_value < 0:
    progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
    header_progression = header+progression_status
  else:
    progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
    header_progression = header+progression_status
  # Merge Header & Song(words+chords)
  full_page=header_progression+str(Progression_song)
  return full_page
dont_need_button=False
try:
  Progression_song = copy.copy(download_song.song)
  header = copy.copy(download_song.header)
  display(InvokeButton('🤩 Get my Song 🎤', download_song),InvokeButton1('  Transform Chords  ', modulazion))
  dont_need_button = True
except:
  with output.use_tags('float_line'):
    display(IPython.display.HTML("<marquee style='width: 50%; color: purple;font-size : 20px;'><b>✌ נא לבחור שיר</b></marquee>"))
 
 
def modulazion():
  #Progression_song = copy.copy(download_song.header)
  #header = copy.copy(download_song.header)
  full_page = modulate_full_page(Progression_song=Progression_song,header=header)
  output.clear(output_tags='download_song_output')
  output.clear(output_tags='modulazion_output')
  with output.use_tags('modulazion_output'):
    #print(sing)
    display(IPython.display.HTML(full_page))
 
def download_song():
  output.clear(output_tags='float_line')
  output.clear(output_tags='download_song_output')
  output.clear(output_tags='modulazion_output')
  on_button_clicked(b=True)
  with output.use_tags('download_song_output'):
    display(IPython.display.HTML(on_button_clicked.full_page))
    download_song.song=on_button_clicked.song
    download_song.header=on_button_clicked.header
    return download_song.song,download_song.header
 
 
 
 
  #display(song_tab,button,IPython.display.HTML(full_page)
if dont_need_button == False:
  display(InvokeButton('🤩 Get my Song 🎤', download_song),InvokeButton1('  Transform Chords  ', modulazion))

In [ ]:
del(first_run)



In [ ]:
class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<style>
    #{callback_id} {{
      background-color: #71A7FF;
      border: none;
      color: white;
      padding: 1%;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      margin: 1%;
      width: 98%;
      cursor: pointer;
      border-radius: 16px;
      box-shadow: 0 9px #CFDFF9;
      font-size :  200%;
    }}

    #{callback_id}:hover {{background-color: #5797FE}}
    #{callback_id}:active {{
      background-color: #71A7FF;
      box-shadow: 0 4px #CFDFF9;
      transform: translateY(4px);
      }}

    </style>
    <button id="{callback_id}">{title}</button>
            <script>
              document.querySelector("#{callback_id}").onclick = (e) => {{
                google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                e.preventDefault();
              }};
            </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

class InvokeButton1(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<style>
    #{callback_id} {{
      background-color: #F688FF;
      border: none;
      color: white;
      padding: 1.3%;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      margin: 1%;
      width: 98%;
      cursor: pointer;
      border-radius: 16px;
      box-shadow: 0 9px #F7E0F9;
      font-size : 200%;
    }}

    #{callback_id}:hover {{background-color: #EF38FE}}
    #{callback_id}:active {{
      background-color: #F688FF;
      box-shadow: 0 4px #F7E0F9;
      transform: translateY(4px);
      }}

    </style>
    <button id="{callback_id}">{title}</button>
            <script>
              document.querySelector("#{callback_id}").onclick = (e) => {{
                google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                e.preventDefault();
              }};
            </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

In [ ]:
from ipywidgets import Button, Layout, jslink, IntText, IntSlider

Invoke_song_button = InvokeButton("🤩 Get my Song 🎤", download_song)._repr_html_()
HTML_song_button = widgets.HTML(Invoke_song_button)

Invoke_chords_button = InvokeButton1('  Transform Chords  ', modulazion)._repr_html_()
HTML_chords_button = widgets.HTML(Invoke_chords_button)
HTML_song_button

HTML(value='<style>\n    #button-6b671f7c-0b74-4f4b-a3a0-ec453ec449fe {\n      background-color: #71A7FF;\n   …

In [ ]:
# Utils widgets
from ipywidgets import Button, Layout, jslink, IntText, IntSlider

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

#song_tab = widgets.Select(options=artist_songs(Search),layout=Layout(width='40%', height='200px',right='True'),rows=20)
song_tab = widgets.Select(options=artist_songs(Search),layout=Layout(width='98%', height='100%',right ='0px',align_self  = 'flex-start'))


top_right_text = widgets.Dropdown(
                  options=[6,5,4,3,2,1,0,-1,-2,-3,-4,-5],
                  value=0,
                  layout=Layout(width='50%', height='60%',align_self  = 'flex-start',right ='0px',object_position='center'),
                  disabled=False)
#top_right_text = IntText(description='Top right', layout=Layout(width='auto', height='auto'))
from ipywidgets import TwoByTwoLayout


full_layout = TwoByTwoLayout(top_left=top_right_text,
                              bottom_left=HTML_chords_button,
                              top_right=song_tab,
                              bottom_right=HTML_song_button,
                              layout=Layout(align_self = 'center'))
full_layout

TwoByTwoLayout(children=(Dropdown(index=6, layout=Layout(align_self='flex-start', grid_area='top-left', height…

In [ ]:
song_tab.value
song_tab.layout.left='0px'
song_tab

Select(layout=Layout(align_self='flex-start', grid_area='top-right', height='100%', left='0px', right='0px', w…

In [ ]:
from ipywidgets import Button, Layout, jslink, IntText, IntSlider,Accordion
accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))

tab_nest = widgets.Tab()
tab_nest.children = [accordion, accordion]
tab_nest.titles = ('An accordion', 'Copy of the accordion')
tab_nest

# Get song index

In [ ]:
from pprint import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import re
import IPython
from google.colab import output
from urllib.request import Request, urlopen

### Find total song number

In [ ]:
song_number_find_url = "https://www.tab4u.com/resultsSimple?tab=songs&type=song&q=*&content=chords&max_chords=0"
req = Request(song_number_find_url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
web_part = soupified.find("td", {"class": "foundTxtTd"})
song_number = [int(i) for i in web_part.get_text().split() if i.isdigit()][0]
song_number

### Scrape tables
https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [ ]:
#%%time
import numpy as np
import string
from tqdm import tqdm
from time import sleep

song_list =[]
#for first_index in np.arange(0,500,100):
for first_index in tqdm(np.arange(0,song_number,100)):
  index_url = f"https://www.tab4u.com/resultsSimple?tab=songs&q=&type=song&cat=&content=chords&max_chords=0&n=100&sort=&s={first_index}"
  req = Request(index_url, headers={'User-Agent': 'Mozilla/5.0'})
  webpage = urlopen(req).read()
  soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
  web_part = soupified.find_all("td", {"class": "songTd1"})
  
  for W in web_part:
    Song_name = W.find('a')['onmouseover'].split(sep=',')[-1].translate(str.maketrans('', '', string.punctuation))
    Artist_name = W.text
    Link2song = "https://www.tab4u.com/"+W.find('a')['href']
    song_list.append([Song_name,Artist_name,Link2song])

df=pd.DataFrame(song_list,columns = ["אמן","שיר","לינק"])
df.to_csv("/content/drive/MyDrive/Colab_yotam/music/SongList.csv",index=False,encoding='utf-8')
df

In [ ]:
# link to the public csv!!!!!
orig_url='https://drive.google.com/file/d/1aDrlFu5eSabjo8CVKAOG6ljfr5OihZLq/view?usp=sharing'

import pandas as pd
import requests
from io import StringIO

file_id = orig_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
text = requests.get(dwn_url)
text.encoding = 'utf-8'
csv_raw = StringIO(text.text)
df = pd.read_csv(csv_raw,encoding='utf-8')
df['אמן'] = df['אמן'].apply(lambda x: x.strip())
df['שיר'] = df['שיר'].apply(lambda x: x.strip())
df.head()

,אמן,שיר,לינק
0,Christina Aguilera,Your Body,https://www.tab4u.com/tabs/songs/3_Christina_A...
1,Maroon 5,One More Night,https://www.tab4u.com/tabs/songs/5_Maroon_5_-_...
2,אבטיפוס,אשה אשה,https://www.tab4u.com/tabs/songs/20_%D7%90%D7%...
3,סינרגיה,צועקים על אהבה,https://www.tab4u.com/tabs/songs/26_%D7%A1%D7%...
4,אבטיפוס,מועקה,https://www.tab4u.com/tabs/songs/30_%D7%90%D7%...
